# Command A Reasoning - Basic Tool Use Example

## Overview
This notebook demonstrates the fundamental concepts of tool use with Cohere's Command A reasoning model. You'll learn how to define tools, make function calls, handle responses, and work with the reasoning traces that show the model's decision-making process.

## Contents
- Setting up basic tool definitions and function mappings
- Understanding tool schemas and parameter specifications
- Making your first tool call with reasoning traces
- Processing tool results and generating responses
- Working with citations and source attribution

In [1]:
import cohere
import json

import os
co = cohere.ClientV2(os.getenv("COHERE_API_KEY"))

In [3]:
model = "command-a-reasoning-08-2025"

In [4]:
def get_weather(location):
    # Implement any logic here
    return [{"temperature": "20°C"}]
    # Return a JSON object string, or a list of tool content blocks e.g. [{"url": "abc.com", "text": "..."}, {"url": "xyz.com", "text": "..."}]


functions_map = {"get_weather": get_weather}

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "gets the weather of a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "the location to get the weather, example: San Francisco.",
                    }
                },
                "required": ["location"],
            },
        },
    },
]


In [10]:
messages = [
    {"role": "user", "content": "What's the weather in Toronto?"}
]


In [11]:
response = co.chat(
    model=model, messages=messages, tools=tools
)

if response.message.tool_calls:
    messages.append(response.message)
    print(response.message.content[0].thinking, "\n")
    print(response.message.tool_calls)


The user is asking for the weather in Toronto. I need to use the available tool 'get_weather' to retrieve this information. The tool requires a 'location' parameter, which the user has provided as Toronto. I'll call the tool with the location set to Toronto. 

[ToolCallV2(id='get_weather_sfnmhqhb8g2p', type='function', function=ToolCallV2Function(name='get_weather', arguments='{"location":"Toronto"}'))]


In [12]:
if response.message.tool_calls:
    for tc in response.message.tool_calls:
        tool_result = functions_map[tc.function.name](
            **json.loads(tc.function.arguments)
        )
        tool_content = []
        for data in tool_result:
            # Optional: the "document" object can take an "id" field for use in citations, otherwise auto-generated
            tool_content.append(
                {
                    "type": "document",
                    "document": {"data": json.dumps(data)},
                }
            )
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tc.id,
                "content": tool_content,
            }
        )

In [13]:
response = co.chat(
    model=model, messages=messages, tools=tools
)

messages.append(response.message)

print(response.message.content[0].thinking)
print("-"*50)
print(response.message.content[1].text)


The user asked for the weather in Toronto. I used the get_weather tool with the location parameter set to "Toronto". The tool returned a result indicating a temperature of 20°C. Since the response only includes the temperature, I'll check if there's additional weather information needed. However, based on the tool's output structure, it seems only the temperature is provided. I'll proceed to inform the user of the current temperature in Toronto.
--------------------------------------------------
The temperature in Toronto is 20°C.


In [14]:
print(response.message.citations)

[Citation(start=30, end=35, text='20°C.', sources=[ToolSource(type='tool', id='get_weather_sfnmhqhb8g2p:0', tool_output={'temperature': '20°C'})], content_index=1, type='TEXT_CONTENT')]


In [15]:
for message in messages:
    print(message, "\n")

{'role': 'user', 'content': "What's the weather in Toronto?"} 

role='assistant' tool_calls=[ToolCallV2(id='get_weather_sfnmhqhb8g2p', type='function', function=ToolCallV2Function(name='get_weather', arguments='{"location":"Toronto"}'))] tool_plan=None content=[ThinkingAssistantMessageResponseContentItem(value=None, type='thinking', thinking="The user is asking for the weather in Toronto. I need to use the available tool 'get_weather' to retrieve this information. The tool requires a 'location' parameter, which the user has provided as Toronto. I'll call the tool with the location set to Toronto.")] citations=None 

{'role': 'tool', 'tool_call_id': 'get_weather_sfnmhqhb8g2p', 'content': [{'type': 'document', 'document': {'data': '{"temperature": "20\\u00b0C"}'}}]} 

role='assistant' tool_calls=None tool_plan=None content=[ThinkingAssistantMessageResponseContentItem(value=None, type='thinking', thinking='The user asked for the weather in Toronto. I used the get_weather tool with the loc